# Анализ результатов
## Оценка качества модели

In [ ]:
import sys
sys.path.append('..')

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from pathlib import Path

import config
from dataset import create_dataloaders
from model import PlantDiseaseClassifier
from utils import calculate_metrics, plot_confusion_matrix

%matplotlib inline

In [ ]:
# Загрузка модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = PlantDiseaseClassifier()
model.load_state_dict(torch.load(config.BEST_MODEL_PATH, map_location=device))
model = model.to(device)
model.eval()
print("Модель загружена")

In [ ]:
# Загрузка тестовых данных
_, _, test_loader = create_dataloaders(config.PLANTVILLAGE_DIR)

In [ ]:
# Получение предсказаний
all_preds = []
all_targets = []
all_probs = []

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        probs = torch.nn.functional.softmax(output, dim=1)
        pred = output.argmax(dim=1)
        
        all_preds.extend(pred.cpu().numpy())
        all_targets.extend(target.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

In [ ]:
# Расчет метрик
metrics = calculate_metrics(all_targets, all_preds)
print("\n=== Метрики классификации ===")
for key, value in metrics.items():
    if isinstance(value, float):
        print(f"{key}: {value:.4f}")

In [ ]:
# Матрица ошибок
cm = confusion_matrix(all_targets, all_preds)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# ROC-кривая
fpr, tpr, _ = roc_curve(all_targets, [p[1] for p in all_probs])
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()